# 杭州萧山区项目`WRF-CMAQ`模拟分析
## 模拟结果分析：*`Process Analysis`*

---
*@author: Evan*\
*@date: 2023-07-13*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from findpoint import findpoint
import ModelEvaLib as me
from namelist import *

## 读取数据

In [2]:
pa = xr.open_dataset(pafile)

ncfile = pa.HADV_O3
HADV   = pa.HADV_O3
ZADV   = pa.ZADV_O3
HDIF   = pa.HDIF_O3
VDIF   = pa.VDIF_O3
DDEP   = pa.DDEP_O3
CHEM   = pa.CHEM_O3

In [5]:
siteloc = pd.read_excel('D:/data/Project_Xiaoshan/杭州站点.xlsx',header=0,index_col=1)

sites = siteloc.index

lono = siteloc['经度']
lato = siteloc['纬度']

print(lono,'\n',lato)

站点
滨江          120.2072
西溪          120.0633
千岛湖(对照点)    119.0260
下沙          120.3461
卧龙桥         120.1269
浙江农大        120.1903
朝晖五区        120.1570
和睦小学        120.1197
临平镇         120.3019
城厢镇         120.2697
云栖          120.0883
镇二中         119.9464
市府大楼        119.7183
消防大队        120.1556
Name: 经度, dtype: float64 
 站点
滨江          30.2111
西溪          30.2747
千岛湖(对照点)    29.6350
下沙          30.3153
卧龙桥         30.2456
浙江农大        30.2692
朝晖五区        30.2897
和睦小学        30.3119
临平镇         30.4183
城厢镇         30.1819
云栖          30.1808
镇二中         30.0494
市府大楼        30.2367
消防大队        30.2864
Name: 纬度, dtype: float64


In [6]:
x_index=[]
y_index=[]
for name in sites:
    out_x, out_y = findpoint(lono[name],lato[name],ncfile)
    x_index.append(out_x)
    y_index.append(out_y)

print(x_index)
print(y_index)

[71, 66, 35, 75, 68, 70, 69, 68, 73, 73, 67, 63, 56, 69]
[72, 75, 50, 76, 74, 75, 75, 76, 80, 71, 71, 66, 73, 75]


# Loop start from here

In [12]:
# =============================================
# Start to loop here 
# =============================================

for i in range(14):

    HADV_sel = HADV.isel(x=x_index[i], y=y_index[i])
    ZADV_sel = ZADV.isel(x=x_index[i], y=y_index[i])
    HDIF_sel = HDIF.isel(x=x_index[i], y=y_index[i])
    VDIF_sel = VDIF.isel(x=x_index[i], y=y_index[i])
    CHEM_sel = CHEM.isel(x=x_index[i], y=y_index[i])
    DDEP_sel = DDEP.isel(x=x_index[i], y=y_index[i])

    nlevel   = 28
    datadict = {}
    outdict  = {}
    vars     = ['HADV','ZADV','HDIF','VDIF','CHEM','DDEP']

    for var in vars:
        datadict[f"{var}_mean"] = np.mean(np.array(locals()[f"{var}_sel"][:,:nlevel]),0)
        outdict[f"{var}_pos"] = np.maximum(0,datadict[f"{var}_mean"])
        outdict[f"{var}_neg"] = np.minimum(0,datadict[f"{var}_mean"])

    # =============================================
    # plot figures
    # =============================================

    h   = np.arange(0,28)

    edgec      = 'white'
    color_hadv = '#4994c4'
    color_zadv = '#a8b092'
    color_hdif = '#00cca6'
    color_vdif = '#e5a84b'
    color_chem = '#e2a2ac'
    color_ddep = '#ad7761'

    fig = plt.figure(figsize=(8,6),dpi=300)
    ax  = fig.subplots(1,1)

    b1=ax.barh(h,outdict['HADV_pos'],color=color_hadv,height=1,edgecolor=edgec)
    ax.barh(h,outdict['HADV_neg'],color=color_hadv,height=1,edgecolor=edgec)

    b2=ax.barh(h,outdict['ZADV_pos'],color=color_zadv,left=outdict['HADV_pos'],height=1,edgecolor=edgec)
    ax.barh(h,outdict['ZADV_neg'],color=color_zadv,left=outdict['HADV_neg'],height=1,edgecolor=edgec)

    b3=ax.barh(h,outdict['HDIF_pos'],color=color_hdif,left=outdict['HADV_pos']+outdict['ZADV_pos'],height=1,edgecolor=edgec)
    ax.barh(h,outdict['HDIF_neg'],color=color_hdif,left=outdict['HADV_neg']+outdict['ZADV_neg'],height=1,edgecolor=edgec)

    b4=ax.barh(h,outdict['VDIF_pos'],color=color_vdif,left=outdict['HADV_pos']+outdict['ZADV_pos']+outdict['HDIF_pos'],height=1,edgecolor=edgec)
    ax.barh(h,outdict['VDIF_neg'],color=color_vdif,left=outdict['HADV_neg']+outdict['ZADV_neg']+outdict['HDIF_neg'],height=1,edgecolor=edgec)

    b5=ax.barh(h,outdict['CHEM_pos'],color=color_chem,left=outdict['HADV_pos']+outdict['ZADV_pos']+outdict['VDIF_pos']+outdict['HDIF_pos'],
                height=1,edgecolor=edgec)
    ax.barh(h,outdict['CHEM_neg'],color=color_chem,left=outdict['HADV_neg']+outdict['ZADV_neg']+outdict['VDIF_neg']+outdict['HDIF_neg'],
                height=1,edgecolor=edgec)

    b6=ax.barh(h,outdict['DDEP_pos'],color=color_ddep,left=outdict['HADV_pos']+outdict['ZADV_pos']+outdict['VDIF_pos']+outdict['HDIF_pos']+outdict['CHEM_pos'],
                height=1,edgecolor=edgec)
    ax.barh(h,outdict['DDEP_neg'],color=color_ddep,left=outdict['HADV_neg']+outdict['ZADV_neg']+outdict['VDIF_neg']+outdict['HDIF_neg']+outdict['CHEM_neg'],
                height=1,edgecolor=edgec)

    ax.set_xlim(-240,240)
    ax.set_ylim(-0.5,27)
    ax.set_xlabel('Concentration ($\mu$$g$/$m^3$)')
    ax.set_ylabel('Height (m)')
    ax.set_xticks(np.arange(-240,241,80))
    ax.set_yticks(np.arange(0,28,3))
    ax.set_yticklabels(['30','100','200','300','500','900','1500','2500','4000','6000'])
    # ax.set_yticklabels(['1000','993','985','972','950','916','850','770','660','520'])

    ax.legend((b5,b1,b2,b3,b4,b6),('CHEM','HADV','ZADV','HDIF','VDIF','DDEP'),bbox_to_anchor=(0.99,-0.1),ncol=6)
    # ax.legend((b5,b1,b2,b3,b4,b6),('CHEM','HADV','ZADV','HDIF','VDIF','DDEP'),loc=2)
    ax.set_title(sites[i],fontdict={'family':'SimSun','size':16})
    
    savepath = '../../figures/PA/IPR/sites/'
    plt.savefig(savepath + str(sites[i]) + '.png')
    print('saving ' + str(sites[i]))
    plt.close()

saving 滨江
saving 西溪
saving 千岛湖(对照点)
saving 下沙
saving 卧龙桥
saving 浙江农大
saving 朝晖五区
saving 和睦小学
saving 临平镇
saving 城厢镇
saving 云栖
saving 镇二中
saving 市府大楼
saving 消防大队
